In [ ]:
# !pip install openai faiss-cpu python-dotenv langchain langchain-openai langchain-community

In [ ]:
import os
import sqlite3
import re
from dotenv import load_dotenv
from IPython.display import display, Markdown
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

load_dotenv()

In [ ]:
# Load embeddings model
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Load FAISS vectorstore saved by build notebook
vectorstore = FAISS.load_local(
    "faiss_index_directory",
    embeddings_model,
    allow_dangerous_deserialization=True
)

# Create retriever for semantic search
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# Connect to SQLite database (if extra info needed)
conn = sqlite3.connect("chunks.db")

In [ ]:
def search_embeddings(query, retriever):
    results = retriever.invoke(query)
    return [(doc.page_content, doc.metadata.get("source", "unknown")) for doc in results]

def get_chatgpt_response(query, relevant_chunks, model="gpt-4o"):
    llm = ChatOpenAI(model=model, temperature=0.0)
    context = "\n\n---\n\n".join(chunk for chunk, _ in relevant_chunks)
    prompt = f"Using the context, answer the question:\n\nContext:\n{context}\n\nQuestion: {query}"
    response = llm.invoke(prompt)
    return response.content

def fix_latex_delimiters(text):
    text = re.sub(r'\\\[(.*?)\\\]', r'$$\1$$', text, flags=re.DOTALL)
    text = re.sub(r'\\\((.*?)\\\)', r'$\1$', text, flags=re.DOTALL)
    return text

In [ ]:
query = """

"""

# Search embeddings using retriever
relevant_chunks = search_embeddings(query, retriever)

# Get ChatGPT-generated response
answer = get_chatgpt_response(query, relevant_chunks)

# Format and display answer clearly
formatted_answer = fix_latex_delimiters(answer)
display(Markdown(formatted_answer))

# Close database connection after querying
conn.close()